## Primer problema

In [1]:
from threading import Thread, Lock
from random import random
import time
lock = Lock()

In [2]:
class Tortuga(Thread):
    def __init__(self, posiciones_tortuga, posiciones_liebre):
        super().__init__()
        self.posiciones_tortuga = posiciones_tortuga
        self.posiciones_liebre = posiciones_liebre
        self.lock = Lock()
        
    def avanzar(self):

        a = random()
        
        # Da un paso de +3 con 50% de probabilidad, -6 con 20% y +1 con 30%.
        if a <= 0.5:
            return 3
        elif a > 0.5 and a <= 0.7:
            return -6
        else:
            return 2
        
    def run(self):
        
        global turno
        
        # Mientras ninguno haya llegado a la meta el ciclo sigue.
        while self.posiciones_tortuga[-1] < 70 and self.posiciones_liebre[-1] < 70:
            # El turno es para sincronizarlos.
            if turno == 0:
                paso = self.avanzar()
                nueva_posicion = self.posiciones_tortuga[-1] + paso
                
                # Nunca puede bajar de 1.
                if nueva_posicion < 1:
                    nueva_posicion = 1

                self.posiciones_tortuga.append(nueva_posicion)

                time.sleep(1)
                turno = (turno + 1)%2

In [3]:
class Liebre(Thread):
    def __init__(self, posiciones_tortuga, posiciones_liebre):
        super().__init__()
        self.posiciones_tortuga = posiciones_tortuga
        self.posiciones_liebre = posiciones_liebre
        self.lock = Lock()
        
    def avanzar(self):
        
        a = random()
        
        # Da un paso de 0 con probabilidad 20%, +9 con 20%, -12 con 10%, 1 con 30% y -2 con 20%.
        if a <= 0.2:
            return 0
        elif a > 0.2 and a <= 0.4:
            return 9
        elif a > 0.4 and a <= 0.5:
            return -12
        elif a > 0.5 and a <= 80:
            return 1
        else:
            return -2
        
    def run(self):
        
        global turno
        
        # Mientras ninguno haya llegado a la meta el ciclo sigue.
        while self.posiciones_tortuga[-1] < 70 and self.posiciones_liebre[-1] < 70:
            # El turno es para sincronizarlos.
            if turno == 1:
                paso = self.avanzar()
                nueva_posicion = self.posiciones_liebre[-1] + paso
                
                # Nunca puede bajar de 1.
                if nueva_posicion < 1:
                    nueva_posicion = 1

                self.posiciones_liebre.append(nueva_posicion)
                
                
                print(' '*self.posiciones_tortuga[-1] + 'T')
                print(' '*nueva_posicion + 'L')
                print('-'*70)

                if self.posiciones_liebre[-1] >= 70 and self.posiciones_tortuga[-1] < 70:
                    print('\n La liebre ha ganado')
                elif self.posiciones_tortuga[-1] >= 70 and self.posiciones_liebre[-1] < 70:
                    print('\n La tortuga ha ganado')
                elif self.posiciones_liebre[-1] >= 70 and self.posiciones_tortuga[-1] >= 70:
                    print('\n Empate')

                time.sleep(1)
                turno = (turno + 1)%2

In [4]:
def Programa1():
    
    posiciones_tortuga = [1]
    posiciones_liebre = [1]
    turno = 0
    
    print('Empieza la carrera \n')

    tortuga = Tortuga(posiciones_tortuga, posiciones_liebre)
    liebre = Liebre(posiciones_tortuga, posiciones_liebre)

    tortuga.start()
    liebre.start()

    tortuga.join()
    liebre.join()
    
    print('\n La carrera ha terminado')

In [5]:
turno = 0
Programa1()

Empieza la carrera 

 T
          L
----------------------------------------------------------------------
   T
           L
----------------------------------------------------------------------
 T
            L
----------------------------------------------------------------------
    T
             L
----------------------------------------------------------------------
      T
                      L
----------------------------------------------------------------------
        T
                               L
----------------------------------------------------------------------
  T
                                        L
----------------------------------------------------------------------
    T
                                         L
----------------------------------------------------------------------
      T
                                         L
----------------------------------------------------------------------
         T
                                     

## Segundo problema

In [6]:
import numpy as np

Sea $A \in M_{n \times p}$ y $B \in M_{p \times m}$, entonces $C = AB \in M_{n \times m}$ donde

$$C_{ij} = \sum_{k = 1}^{p} A_{ip}B_{pj}$$

In [7]:
def multiplicacion_matrices_secuencial(A, B):
    
    """
    Esta función hace la multiplicación de dos matrices directo de la definición.
    """
    
    n, p1 = A.shape
    p2, m = B.shape
    
    C = np.zeros((n,m))
    
    if p1 != p2:
        return 'Matrices incompatibles'
    else:
        for i in range(n):
            for j in range(m):
                C[i,j] = sum([A[i,k]*B[k, j] for k in range(p1)])
                
    return C

Luego hagamos 4 hilos, cada uno calculará lo siguiente:

\begin{align}
C_{1} \rightarrow C_{ij} = \sum_{k = 1}^{p} A_{ik}B_{kj} \quad \text{donde} \quad i \leq \left \lfloor{\frac{n}{2}}\right \rfloor, \ j \leq \left \lfloor{\frac{m}{2}}\right \rfloor \\
C_{2} \rightarrow C_{ij} = \sum_{k = 1}^{p} A_{ik}B_{kj} \quad \text{donde} \quad i \leq \left \lfloor{\frac{n}{2}}\right \rfloor, \ j > \left \lceil{\frac{m}{2}}\right \rceil \\
C_{3} \rightarrow C_{ij} = \sum_{k = 1}^{p} A_{ik}B_{kj} \quad \text{donde} \quad i > \left \lceil{\frac{n}{2}}\right \rceil, \ j \leq \left \lfloor{\frac{m}{2}}\right \rfloor \\
C_{4} \rightarrow C_{ij} = \sum_{k = 1}^{p} A_{ik}B_{kj} \quad \text{donde} \quad i > \left \lceil{\frac{n}{2}}\right \rceil, \ j > \left \lceil{\frac{m}{2}}\right \rceil
\end{align}

Por tanto, cada hilo calcula una parte de la matriz resultado.

In [8]:
def multiplicacion_subconjunto_matriz(A, B, C, parte):
    
    """
    Esta función hace la multiplicación de dos matrices A,B por bloques y va llenando la matriz
    C con el resultado.
    """
        
    n, p1 = A.shape
    p2, m = B.shape
    
    if parte == 1:
        for i in range(int(np.floor(n/2))):
            for j in range(int(np.floor(m/2))):
                C[i,j] = sum([A[i,k]*B[k, j] for k in range(p1)])
                
    elif parte == 2:
        for i in range(int(np.floor(n/2))):
            for j in range(int(np.ceil(m/2)), m):
                C[i,j] = sum([A[i,k]*B[k, j] for k in range(p1)])
                
    elif parte == 3:
        for i in range(int(np.ceil(n/2)), n):
            for j in range(int(np.floor(m/2))):
                C[i,j] = sum([A[i,k]*B[k, j] for k in range(p1)])
                
    elif parte == 4:
        for i in range(int(np.ceil(n/2)), n):
            for j in range(int(np.ceil(m/2)), m):
                C[i,j] = sum([A[i,k]*B[k, j] for k in range(p1)])

In [9]:
def Programa2(A, B):
    
    """
    Esta función compara el tiempo que toma hacer una multiplicación de matrices de 
    forma secuencial y de forma concurrente.
    """
    
    n, q1 = A.shape
    q2, m = B.shape
    
    C = np.zeros((n, m))
    
    t1 = time.time()
    
    m1 = Thread(target=multiplicacion_subconjunto_matriz, args=(A, B, C, 1))
    m2 = Thread(target=multiplicacion_subconjunto_matriz, args=(A, B, C, 2))
    m3 = Thread(target=multiplicacion_subconjunto_matriz, args=(A, B, C, 3))
    m4 = Thread(target=multiplicacion_subconjunto_matriz, args=(A, B, C, 4))

    m1.start()
    m2.start()
    m3.start()
    m4.start()

    m1.join()
    m2.join()
    m3.join()
    m4.join()
    
    print('Multiplicar las matrices concurrentemente tomó : ' + str(time.time() - t1) + ' segundos \n')
    print(C)
    print('\n')
    
    t2 = time.time()

    result = multiplicacion_matrices_secuencial(A,B)

    print('Multiplicar las matrices secuencialmente tomó : ' + str(time.time() - t2) + ' segundos \n')
    print(result)

En lo siguiente vemos que dan las mismas matrices, pero si la dimensión de las matrices es pequeña no nos conviene hacer la multiplicación concurrente pues tardamos más. Específicamente, de manera secuencial es más de 30 veces más rápido que de manera concurrente.  

In [19]:
A = np.random.rand(4, 4)
B = np.random.rand(4, 4)

Programa2(A,B)

Multiplicar las matrices concurrentemente tomó : 0.0073969364166259766 segundos 

[[1.0459873  0.57886527 0.82760051 1.04996277]
 [1.23478669 0.65595752 0.91019747 1.2001057 ]
 [1.44068912 0.86204888 1.26601263 1.50412845]
 [1.45854128 1.01110958 1.46746391 1.26955752]]


Multiplicar las matrices secuencialmente tomó : 0.00021767616271972656 segundos 

[[1.0459873  0.57886527 0.82760051 1.04996277]
 [1.23478669 0.65595752 0.91019747 1.2001057 ]
 [1.44068912 0.86204888 1.26601263 1.50412845]
 [1.45854128 1.01110958 1.46746391 1.26955752]]


Con una matriz de 500 x 500 (1000 x 1000 hubiera tomado demasiado) también es mejor de manera secuencial, pero ahora sólo es 1.5 más rápido que de manera concurrente. Podemos esperar que para matrices de mucho mayor dimensión la multiplicación de matrices de manera concurrente sea más rápida que la secuencial.

In [23]:
A = np.random.rand(500, 500)
B = np.random.rand(500, 500)

Programa2(A,B)

Multiplicar las matrices concurrentemente tomó : 300.2573890686035 segundos 

[[132.95438685 130.29347793 126.9920408  ... 133.66119665 125.31616911
  130.33751561]
 [130.27021136 124.36097164 123.05744799 ... 126.28495495 121.84292941
  125.30160976]
 [133.09185951 129.59882815 127.62152847 ... 131.17638134 124.35742226
  129.27594535]
 ...
 [132.97020967 129.11839924 125.38519214 ... 125.75795958 121.11051882
  127.69920735]
 [130.87145111 127.44108071 120.06144744 ... 127.88272167 122.96203645
  122.81323938]
 [130.55118151 126.64060189 123.57394712 ... 126.09714899 120.23858952
  127.93688376]]


Multiplicar las matrices secuencialmente tomó : 194.9183588027954 segundos 

[[132.95438685 130.29347793 126.9920408  ... 133.66119665 125.31616911
  130.33751561]
 [130.27021136 124.36097164 123.05744799 ... 126.28495495 121.84292941
  125.30160976]
 [133.09185951 129.59882815 127.62152847 ... 131.17638134 124.35742226
  129.27594535]
 ...
 [132.97020967 129.11839924 125.38519214 ... 125.